In [3]:
import tensorflow as tf

# GPU 사용 설정
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # GPU 메모리 증가 허용
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus, 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        print(e)


### 1. **데이터셋 로드**
# 우선, 이미지 데이터를 불러옵니다. 이미지를 다루는 경우, 일반적으로 픽셀 값을 특성으로 변환하고 각 이미지를 평탄화(flatten)해야 합니다. 가령, 64x64 RGB 이미지를 1차원 벡터로 변환합니다.

#### 예시 (이미지 데이터 로드):
# 예시로 CIFAR-10 데이터를 로드
import numpy as np
from tensorflow.keras.datasets import cifar10

# 데이터 로드 (이미지 크기: 32x32x3)
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = cifar10.load_data()

# 0 또는 1의 레이블로 변환 (예를 들어 고양이와 나머지 구분)
Y_train = (Y_train_orig == 3).astype(int)  # 고양이가 3번 클래스
Y_test = (Y_test_orig == 3).astype(int)


### 2. **데이터 전처리 및 평탄화 (Flattening)**
# 이미지를 신경망에 입력하기 위해 2D 이미지 데이터를 1차원 벡터로 변환합니다. 또한, 픽셀 값을 0과 1 사이로 정규화하여 학습이 더 잘되도록 합니다.

# 훈련 데이터 평탄화
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T

# 데이터 정규화 (0 ~ 1 범위로 만들기 위해 255로 나눔)
X_train = X_train_flatten / 255.
X_test = X_test_flatten / 255.

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")


### 3. **가중치와 편향 초기화**
# 로지스틱 회귀 모델에서 가중치 `w`와 편향 `b`를 0으로 초기화합니다.
w = np.zeros((X_train.shape[0], 1))  # 특성 수 만큼 가중치 초기화
b = 0.0  # 편향 초기화

### 4. **전방 전파 (Forward Propagation)**
# `w`와 `b`를 사용해 전방 전파를 계산하고, **시그모이드 함수**를 통해 예측값을 계산합니다.

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def propagate(w, b, X, Y):
    m = X.shape[1]
    
    # 전방 전파
    A = sigmoid(np.dot(w.T, X) + b)  # 활성화 값 A 계산
    cost = (-1 / m) * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))  # 비용 함수 계산
    
    # 역방향 전파 (경사도 계산)
    dw = (1 / m) * np.dot(X, (A - Y).T)
    db = (1 / m) * np.sum(A - Y)
    
    cost = np.squeeze(cost)
    grads = {"dw": dw, "db": db}
    
    return grads, cost

### 5. **경사 하강법을 통한 최적화**
#전방 전파와 역방향 전파를 통해 계산한 경사도를 사용해 가중치와 편향을 업데이트합니다.

def optimize(w, b, X, Y, num_iterations, learning_rate):
    costs = []
    
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        
        # 파라미터 업데이트
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # 비용 저장
        if i % 100 == 0:
            costs.append(cost)
            print(f"Cost after iteration {i}: {cost}")
    
    params = {"w": w, "b": b}
    grads = {"dw": dw, "db": db}
    
    return params, grads, costs

### 6. **예측 (Prediction)**
# 학습된 가중치와 편향을 사용해 테스트 데이터와 훈련 데이터에 대한 예측을 수행합니다.

def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
    
    return Y_prediction

### 7. **모델 학습 및 예측**
# 모델을 학습하고, 테스트 데이터와 훈련 데이터에 대해 예측을 수행합니다.


# 학습 (경사 하강법 최적화)
num_iterations = 2000
learning_rate = 0.005
params, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate)

# 최적화된 가중치와 편향
w = params["w"]
b = params["b"]

# 예측 수행
Y_prediction_test = predict(w, b, X_test)
Y_prediction_train = predict(w, b, X_train)

# 정확도 계산
train_accuracy = 100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100
test_accuracy = 100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100

print(f"Train accuracy: {train_accuracy}%")
print(f"Test accuracy: {test_accuracy}%")

### 전체 과정 요약:
# 1. **데이터셋 로드**: 훈련과 테스트 데이터를 로드합니다.
# 2. **데이터 전처리**: 이미지 데이터를 1차원 벡터로 변환하고 정규화합니다.
# 3. **가중치와 편향 초기화**: 가중치 `w`와 편향 `b`를 0으로 초기화합니다.
# 4. **전방 전파(Forward Propagation)**: 시그모이드 함수를 사용해 예측을 계산합니다.
# 5. **비용 함수 계산**: 로지스틱 손실 함수로 비용을 계산합니다.
# 6. **경사 하강법(Backward Propagation)**: 경사도를 사용해 가중치와 편향을 업데이트합니다.
# 7. **예측**: 학습된 모델로 테스트 데이터와 훈련 데이터를 예측하고, 정확도를 확인합니다.

# 이 과정을 통해 **로지스틱 회귀** 모델을 사용해 이미지를 분류하고, 모델이 얼마나 정확한지 확인할 수 있습니다.

Physical devices cannot be modified after being initialized
X_train shape: (3072, 50000)
X_test shape: (3072, 10000)
Cost after iteration 0: 34657.35902819863


$$
z=w 
T
 X+b
 $$
선형 함수 z를 활성화 함수를 이용해 비선형 함수로 만든다

In [ ]:
import tensorflow as tf

# 1. 데이터 로드 (cifar10) 및 전처리
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = tf.keras.datasets.cifar10.load_data()

# 고양이만 분류하는 이진 분류 문제로 변환 (3번 클래스)
Y_train = (Y_train_orig == 3).astype(int)
Y_test = (Y_test_orig == 3).astype(int)

# 이미지 평탄화 및 정규화
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1) / 255.
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1) / 255.

# 2. TensorFlow 모델 생성
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_flatten.shape[1],)),
    tf.keras.layers.Dense(1, activation='sigmoid')  # 로지스틱 회귀
])

# 3. 모델 컴파일 (손실 함수 및 최적화 알고리즘 설정)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              loss='binary_crossentropy',  # 이진 분류를 위한 손실 함수
              metrics=['accuracy'])

# 4. 모델 학습
history = model.fit(X_train_flatten, Y_train, epochs=10, batch_size=64, validation_data=(X_test_flatten, Y_test))

# 5. 결과 확인
train_loss, train_accuracy = model.evaluate(X_train_flatten, Y_train)
test_loss, test_accuracy = model.evaluate(X_test_flatten, Y_test)

print(f"Train accuracy: {train_accuracy * 100:.2f}%")
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import cifar10

# 데이터 로드 (이미지 크기: 32x32x3)
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = cifar10.load_data()

# CIFAR-10 데이터셋의 클래스 (0~9)
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# 5개의 이미지와 라벨을 표시
plt.figure(figsize=(10,10))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train_orig[i])
    plt.xlabel(class_names[int(Y_train_orig[i])])  # 해당 이미지를 나타내는 클래스명
plt.show()


 Z는 시그모이드 함수의 입력값 (특정 층에서 가중치와 바이어스를 적용한 선형 변환의 결과)
 a^[1]는 z에 활성화 함수를 적용한 결과값 


매개변수(가중치)를 모두 0으로 초기화하면 안 되는 이유는 신경망 학습에서 **대칭성 깨기(symmetry breaking)**가 필요하기 때문
- 모든 가중치를 0으로 초기화하면, 모든 뉴런이 동일한 기울기를 가지게 되며, 모든 뉴런이 같은 학습을 해서 다른 패턴을 학습하지 못한다
- 경사 하강법의 문제: 역전파시 경사도를 계산할 때 뉴런에 대해 동일한 값이 전달된다. 이 경우 네트워크가 복잡한 패턴을 학습하지 못하며, 뉴련간의 차별성이 없어진다

따라서 무작위 값으로 초기화하여 뉴런들이 서로 다른 학습을 할 수 있게 해야 한다.
 **작은 무작위 값으로 ** 초기화

신경망의 활성화 값과 관련된 표기법
- 𝑎[2](12)는 12번째 훈련 예제에서 2번째 층의 활성화 벡터를 나타낸다.
- X는 각 열이 하나의 훈련 예제
- 𝑎4[2]는 2번째 층의 4번째 뉴런에 의한 활성화 출력.

$a_i^{[l]}$
 : 여기서
**
𝑙
**은 층 번호를 의미합니다. 즉, 몇 번째 층의 활성화 값인지를 나타냅니다. <br>
**
𝑖
**는 해당 층에서 몇 번째 뉴런에 해당하는 활성화 값인지를 나타냅니다. <br>
$a_i^{[l]}(m)$의 형태는 m번째 훈련 예제에 대한 해당 뉴련의 활성화 값을 의미한다.

In [3]:
import numpy as np
A = np.random.randn(4, 3)
print(A)
# A는 다음과 같은 4x3 배열을 가질 수 있습니다.
# [[ 0.5,  1.2, -0.3],
#  [-1.1,  0.4,  0.8],
#  [ 0.9, -0.6,  1.5],
#  [ 1.0,  1.1,  0.2]]

B = np.sum(A, axis=1, keepdims=True)
print(B)
# B는 각 행의 합계를 유지한 (4, 1) 배열입니다.
# [[ 1.4],
#  [ 0.1],
#  [ 1.8],
#  [ 2.3]]

'keepdims=True는 차원을 유지하라는 옵션, 기본적으로 np.sum()은 차원을 제거한다.'
B.shape

(4, 1)

In [ ]:
import numpy as np

# 입력 데이터: 4개의 입력 뉴런 (x1, x2, x3, x4)
X = np.random.randn(4, 1)  # (4, 1) 크기의 입력 데이터

# 첫 번째 은닉층: 2개의 뉴런 (a1_1, a1_2)
# W[1]: (2, 4) 크기의 가중치 행렬 (2개의 출력 뉴런, 4개의 입력 뉴런)
W1 = np.random.randn(2, 4)

# b[1]: (2, 1) 크기의 편향 벡터 (2개의 출력 뉴런에 대응하는 편향)
b1 = np.random.randn(2, 1)

# 첫 번째 은닉층 활성화 값 계산 (a1)
Z1 = np.dot(W1, X) + b1  # Z1 = W1 * X + b1
a1 = np.tanh(Z1)         # 활성화 함수로 tanh를 사용 (원하는 함수로 변경 가능)

# 출력층: 1개의 뉴런 (a2_1)
# W[2]: (1, 2) 크기의 가중치 행렬 (1개의 출력 뉴런, 2개의 입력 뉴런)
W2 = np.random.randn(1, 2)

# b[2]: (1, 1) 크기의 편향 벡터 (1개의 출력 뉴런에 대응하는 편향)
b2 = np.random.randn(1, 1)

# 출력층 활성화 값 계산 (a2)
Z2 = np.dot(W2, a1) + b2  # Z2 = W2 * a1 + b2
a2 = 1 / (1 + np.exp(-Z2))  # 활성화 함수로 시그모이드 사용

# 최종 출력
y_hat = a2  # 예측값 y_hat

- X: 입력 데이터, 크기는 **(4, 1)**이며 4개의 입력 뉴런을 의미합니다.
- W1: 첫 번째 은닉층의 가중치, 크기는 **(2, 4)**입니다. 이는 2개의 출력 뉴런과 4개의 입력 뉴런을 의미합니다.
- b1: 첫 번째 은닉층의 편향, 크기는 **(2, 1)**입니다. 이는 각 출력 뉴런에 대응하는 편향입니다.
- a1: 첫 번째 은닉층의 활성화 값, tanh 활성화 함수를 사용했습니다.
- W2: 출력층의 가중치, 크기는 **(1, 2)**입니다. 이는 1개의 출력 뉴런과 2개의 입력 뉴런을 의미합니다.
- b2: 출력층의 편향, 크기는 **(1, 1)**입니다.
- y_hat: 최종 출력 값으로, 시그모이드 함수를 사용하여 이진 분류 문제의 예측 값을 계산했습니다.

가중치 행렬의 크기는 **(출력 뉴런 수, 입력 뉴런 수)**이므로 <br>
입력: 4개의 입력 뉴런 (즉, 𝑥1,𝑥2,𝑥3,𝑥4x)<br>
출력: 2개의 뉴런<br>
𝑊[1] 의 크기 = (2,4) <br>
<br><br>
편향은 출력 뉴런의 수에 하나씩 대응되므로, 편향 벡터의 크기는 (출력 뉴런 수, 1)<br>
b[1]  의 크기= (2,1)<br>
<br>


입력 뉴런: 𝑥1, 𝑥2 즉 입력 뉴런은 2개<br>
첫 번째 은닉층의 뉴런: a1[1] ,a2[1] ,a3[1] ,a4[1]<br>
​<br><br>
Z[1]의 크기: 첫 번째 은닉층의 뉴런에 대한 가중합을 의미합니다.<br>
입력 뉴런이 2개이고, 첫번째 은닉층의 뉴런이 4개 이므로 Z[1]의 크기는 (4, m)<br>


A[1]의 크기 = Z[1]과 같은 크기

In [ ]:
# 데이터 시각화 (예: scatter plot)
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral)
plt.show()

In [ ]:
# 임의의 데이터를 생성하여 크기 확인
import numpy as np

X = np.random.randn(2, 400)  # (2, 400) 크기의 임의의 데이터 생성
Y = np.random.randn(1, 400)  # (1, 400) 크기의 임의의 데이터 생성

shape_X = X.shape
shape_Y = Y.shape
m = X.shape[1]  # 훈련 예제의 수는 열 개수로 계산

print('The shape of X is: ' + str(shape_X))
print('The shape of Y is: ' + str(shape_Y))
print('I have m = %d training examples!' % (m))

In [ ]:
# 명확한 데이터가 이미 로드된 상태에서 shape 확인
X, Y = load_planar_dataset()  # 데이터셋 로드 (예시)
shape_X = X.shape             # X의 형태를 가져옴
shape_Y = Y.shape             # Y의 형태를 가져옴
m = X.shape[1]                # 훈련 예제의 개수는 X의 열 개수로 정의됨

print('The shape of X is: ' + str(shape_X))  # X의 형태 출력
print('The shape of Y is: ' + str(shape_Y))  # Y의 형태 출력
print('I have m = %d training examples!' % (m))  # 훈련 예제 개수 출력

X.shape[0]는 입력 데이터의 특징 수(입력층의 뉴런 개수)를 의미합니다.


Y.shape[0]는 출력 데이터의 크기(출력층의 뉴런 개수)를 의미합니다.


은닉층은 문제에서 하드코딩하여 4로 설정합니다.

In [ ]:
'Argument'
# n_x -- 입력층의 크기 (입력층의 노드 수, X의 첫번째 차원)
# n_h -- 은닉층의 크기 (은닉층의 노드 수)
# n_y -- 출력층의 크기 (출력층의 노드 수)

W1 = np.random.randn(n_h, n_x) * 0.01  # 작은 랜덤 값으로 초기화
b1 = np.zeros((n_h, 1))                # 0으로 초기화
W2 = np.random.randn(n_y, n_h) * 0.01  # 작은 랜덤 값으로 초기화
b2 = np.zeros((n_y, 1))                # 0으로 초기화

**W1**: 입력층과 은닉층 간의 가중치 행렬이며, 크기는 (n_h, n_x)입니다. np.random.randn(n_h, n_x) * 0.01을 사용하여 작은 랜덤 값으로 초기화합니다.<br>
**b1**: 은닉층의 편향 벡터, 크기는 (n_h, 1), np.zeros((n_h, 1))를 사용하여 0으로 초기화한다.<br>
**W2**: 은닉층과 출력층 간의 가중치 행렬, 크기는 (n_y, n_h). np.random.randn(n_y, n_h) * 0.01을 사용하여 작은 랜덤 값으로 초기화<br>
**b2**: 출력층의 편향 벡터. 크기는 (n_y, 1). np.zeros((n_y, 1))를 사용하여 0으로 초기화한다

In [ ]:
parameters = {
    "W1": np.random.randn(3, 2),  # 첫 번째 층의 가중치
    "b1": np.zeros((3, 1)),       # 첫 번째 층의 편향
    "W2": np.random.randn(1, 3),  # 두 번째 층의 가중치
    "b2": np.zeros((1, 1))        # 두 번째 층의 편향
}

# parameters 딕셔너리에서 가중치와 편향을 가져오는 방식
W1 = parameters["W1"]
b1 = parameters["b1"]

In [ ]:
# parameters 딕셔너리 내용:
{
    'W1': array([[ 0.1, -1.2],
                 [ 1.3,  0.5],
                 [-0.4,  0.2]]),
    'b1': array([[0.],
                 [0.],
                 [0.]]),
    'W2': array([[ 0.3, -0.6,  0.7]]),
    'b2': array([[0.]])
}

# W1 가중치 배열:
[[ 0.1 -1.2]
 [ 1.3  0.5]
 [-0.4  0.2]]

In [3]:
# 딕셔너리 생성
import numpy as np

# 배열 초기화
W1 = np.random.randn(4, 3)  # 4x3 형태의 가중치 배열
b1 = np.zeros((4, 1))       # 4x1 형태의 편향 배열
W2 = np.random.randn(1, 4)  # 1x4 형태의 가중치 배열
b2 = np.zeros((1, 1))       # 1x1 형태의 편향 배열

# 딕셔너리 생성
parameters = {
    "W1": W1,  # 가중치 배열을 딕셔너리에 저장
    "b1": b1,  # 편향 배열을 딕셔너리에 저장
    "W2": W2,  # 가중치 배열
    "b2": b2   # 편향 배열
}

# 딕셔너리 출력
print(parameters)

{'W1': array([[-1.34655   ,  1.00616122,  0.42421941],
       [-0.35107087,  0.19093117,  0.02351876],
       [ 0.46988999,  1.2635511 ,  0.32752464],
       [-0.84803007, -0.1987923 ,  0.32245407]]), 'b1': array([[0.],
       [0.],
       [0.],
       [0.]]), 'W2': array([[ 0.66591241, -0.35810343,  1.03054955,  0.22810726]]), 'b2': array([[0.]])}


**Z1, A1, Z2, A2**는 신경망의 전방 전파(Forward Propagation) 단계에서 각 층에서 계산되는 값

In [ ]:
Z1 = np.dot(W1, X) + b1
# Z1은 첫 번째 은닉층의 선형 변환 값을 계산한 결과
# W1은 첫 번째 은닉층의 가중치 행렬
# X는 입력 데이터
# 이 둘을 행렬 곱(np.dot)한 후에 편향 b1을 더한 값이 Z1 / 아직 활성화 함수가 적용되지 않은 모습

In [ ]:
A1 = np.tanh(Z1)
# A1은 Z1에 활성화 함수인 tanh를 적용한 값. -1 과 1사이로 변환하는 비선형 함수

In [ ]:
Z2 = np.dot(W2, A1) + b2
# Z2는 첫 번째 출력층의 선형 변환 값을 계산한 결과
# W2는 두 번째 층의 가중치, A1은 첫 번째 입력값을 출력값, 이 둘을 행렬 곱 한 후에 편향 b2를 더한 값이 Z2

In [ ]:
A2 = sigmoid(Z2)
# A2는 Z2에 시그모이드 활성화 함수를 적용한 값

손실 함수 계산법
$$
𝐽=−1𝑚∑𝑖=1𝑚(𝑦(𝑖)log(𝑎[2](𝑖))+(1−𝑦(𝑖))log(1−𝑎[2](𝑖)))
$$

In [ ]:
logprobs = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))
cost = - np.sum(logprobs) / m
cost = float(np.squeeze(cost))

In [ ]:
logprobs = np.dot(Y, np.log(A2).T) + np.dot((1 - Y), np.log(1 - A2).T)
cost = - logprobs / m  # 평균화된 비용

In [4]:
# 파이토치 손실 함수 계산
import torch
import torch.nn as nn

# 교차 엔트로피 손실 함수
loss_fn = nn.BCELoss()

# 예측값과 실제값
predictions = torch.sigmoid(torch.randn(10, 1))  # 예측값
targets = torch.rand(10, 1)  # 실제값

# 손실 계산
loss = loss_fn(predictions, targets)
print(loss)

tensor(0.8776)


In [5]:
# 텐서플로 손실 함수 계산
import tensorflow as tf

# 교차 엔트로피 손실 함수
loss_fn = tf.keras.losses.BinaryCrossentropy()

# 예측값과 실제값
predictions = tf.random.normal([10, 1])  # 예측값
targets = tf.random.uniform([10, 1], minval=0, maxval=1)  # 실제값

# 손실 계산
loss = loss_fn(targets, tf.sigmoid(predictions))
print(loss)

tf.Tensor(0.7014014, shape=(), dtype=float32)


**손실 함수는 예측값과 실제값 사이의 차이를 측정하여 모델의 예측 성능을 평가한다**

# 4.5 - 역전파(Backpropagation) 구현
전방 전파(Forward Propagation) 동안 계산된 **캐시(cache)**를 사용하여 이제 역전파를 구현할 수 있습니다.

역전파는 **기울기(gradient)**를 계산하는 과정으로, 각 가중치와 편향에 대한 오차의 기울기를 계산하여 신경망 학습에서 가중치와 편향을 업데이트할 수 있게 합니다. <br>
슬라이드에서 오른쪽에 나와 있는 방정식들은 역전파 계산에 필요한 수식을 요약한 것입니다. 이 방정식을 사용하여 각 층의 가중치와 편향의 기울기를 계산합니다.<br>
전방 전파에서 계산된 값들을 사용하여 역전파에서 필요한 기울기를 구하고, 이를 기반으로 가중치와 편향을 업데이트합니다.

In [ ]:
# 캐시 딕셔너리 예시
cache = {
    "Z1": Z1,  # 첫 번째 레이어의 선형 변환 결과
    "A1": A1,  # 첫 번째 레이어의 활성화 함수 적용 결과
    "Z2": Z2,  # 두 번째 레이어의 선형 변환 결과
    "A2": A2   # 최종 출력 (예측값)
}

parameters는 **전방 전파(Forward Propagation)**를 수행하기 위해 필요한 **가중치(W)**와 편향(b) 같은 모델의 매개변수를 저장한 딕셔너리입니다.<br> 
이 값들은 학습 과정에서 조정되며, 모델이 데이터를 학습할 수 있게 돕는 중요한 요소들<br>

cache는 **역전파(Backpropagation)**를 수행할 때 사용하기 위해 전방 전파에서 계산된 중간 값들(예: Z1, A1, Z2, A2)을 저장하는 딕셔너리입니다. <br>
이 값들은 기울기 계산을 위해 필요하며, 학습 과정에서 파라미터를 업데이트하는 데 사용됩니다.

전방 전파(Forward Propagation): parameters에서 가중치와 편향을 불러와 계산을 수행하고, 그 결과를 cache에 저장합니다. <br>
역전파(Backpropagation): cache에 저장된 값들을 사용해 기울기를 계산하고, 이 기울기를 바탕으로 parameters에 저장된 가중치와 편향을 업데이트합니다.

In [ ]:
# 역전파 구현
'1. parameters에서 W1, W2 가져오기'
W1 = parameters["W1"]
W2 = parameters["W2"]

'2. cache에서 A1, A2 가져오기'
A1 = cache["A1"]
A2 = cache["A2"]

'3. 기울기 계산'
dZ2 = A2 - Y # 출력값과 실제 값의 차이를 계산.
dW2 = (1 / m) * np.dot(dZ2, A1.T) # 두 번째 층의 가중치에 대한 기울기 계산.
db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True) # 두 번째 층의 편향에 대한 기울기 계산.
dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2)) # 첫 번째 층에서의 활성화 함수 미분을 고려한 오차 계산.

dW1 = (1 / m) * np.dot(dZ1, X.T) # 첫 번째 층의 가중치에 대한 기울기 계산.
db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True) # 첫 번째 층의 편향에 대한 기울기 계산.

'모든 기울기를 grads 딕셔너리에 저장하고 반환합니다.'

In [ ]:
'모든 기울기를 grads 딕셔너리에 저장하고 반환합니다.'
grads = {"dW1": dW1,  # W1의 기울기
         "db1": db1,  # b1의 기울기
         "dW2": dW2,  # W2의 기울기
         "db2": db2}  # b2의 기울기

# 4-6 경사 하강법 Gradient Descent 
이 공식은 파라미터 θ를 조금씩 업데이트하여 손실 함수 J(θ)가 최소화되도록 하는 과정입니다. <br>
기울기를 따라서 내려가는 경사 하강법을 사용해 파라미터를 업데이트하며, 파라미터를 계속해서 수정하면서 최적의 값을 찾아갑니다.

In [ ]:
'1. 파라미터와 기울기 가져오기'
parameters, grads = update_parameters_test_case()
# parameters에는 신경망의 가중치 W1, W2와 편향 b1, b2가 포함되어 있습니다.
# grads에는 경사 하강법에서 계산된 dW1, dW2, db1, db2라는 기울기가 들어 있습니다.

In [ ]:
'2. 경사 하강법을 통한 파라미터 업데이트'
parameters = update_parameters(parameters, grads)
# 이전에 저장된 parameters 딕셔너리로부터 가중치와 편향을 복사합니다 (copy.deepcopy()).
# grads에서 기울기 값을 가져옵니다.

'경사 하강법을 통해 새로운 가중치와 편향을 계산하고, 이 값을 딕셔너리에 저장합니다.'
W1 = W1 - learning_rate * dW1
b1 = b1 - learning_rate * db1
W2 = W2 - learning_rate * dW2
b2 = b2 - learning_rate * db2
# 학습률 learning_rate에 따라 기울기만큼 파라미터를 조정합니다

'업데이트된 parameters를 반환합니다.'

`for`문을 사용하는 이유는 신경망 학습 과정에서 경사 하강법(Gradient Descent)이 **여러 번 반복**되어야 하기 때문입니다. **경사 하강법**은 반복적인 과정을 통해 손실 함수의 기울기를 계산하고, 가중치와 편향을 업데이트하는 과정입니다. 따라서 여러 번의 반복이 필요하며, 이를 위해 `for`문을 사용하여 **지정된 횟수만큼** 학습을 진행하게 됩니다.

이전에 `for`나 `while`문을 사용하지 말라는 조건은 특정 문제에 대한 제약이었을 수 있지만, 신경망 학습 과정은 기본적으로 반복이 필요한 작업이므로 여기서는 예외적으로 `for`문을 사용하여 학습을 진행합니다.

### 문제 풀이:

이제 `nn_model()` 함수의 문제를 하나씩 해결해 보겠습니다.

```python
# GRADED FUNCTION: nn_model

def nn_model(X, Y, n_h, num_iterations=10000, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)  # 랜덤 시드 설정
    n_x = layer_sizes(X, Y)[0]  # 입력층 크기
    n_y = layer_sizes(X, Y)[2]  # 출력층 크기
    
    # 파라미터 초기화
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    # 경사 하강법 루프
    for i in range(0, num_iterations):
        
        # 1. 전방 전파
        A2, cache = forward_propagation(X, parameters)
        
        # 2. 비용 함수 계산
        cost = compute_cost(A2, Y)
        
        # 3. 역전파
        grads = backward_propagation(parameters, cache, X, Y)
        
        # 4. 파라미터 업데이트
        parameters = update_parameters(parameters, grads)
        
        # 매 1000번의 반복마다 비용 출력
        if print_cost and i % 1000 == 0:
            print("Cost after iteration %i: %f" % (i, cost))

    return parameters
```

### 코드 설명:

1. **파라미터 초기화** (`initialize_parameters()`):
   ```python
   parameters = initialize_parameters(n_x, n_h, n_y)
   ```
   여기서 입력층(`n_x`), 은닉층(`n_h`), 출력층(`n_y`)의 크기를 기반으로 신경망의 가중치와 편향을 초기화합니다.

2. **경사 하강법 루프**:
   - 학습 과정은 `num_iterations` 만큼 반복되며, 각 반복마다 신경망의 가중치와 편향이 업데이트됩니다.

3. **전방 전파** (`forward_propagation()`):
   ```python
   A2, cache = forward_propagation(X, parameters)
   ```
   - 입력 데이터 `X`와 파라미터를 사용하여 전방 전파를 통해 **출력 값**과 중간 계산 값(`cache`)을 계산합니다.

4. **비용 함수 계산** (`compute_cost()`):
   ```python
   cost = compute_cost(A2, Y)
   ```
   - 예측된 출력 값 `A2`와 실제 값 `Y`를 사용하여 비용(손실)을 계산합니다.

5. **역전파** (`backward_propagation()`):
   ```python
   grads = backward_propagation(parameters, cache, X, Y)
   ```
   - 전방 전파에서 계산된 중간 값과 비용을 사용해 기울기(gradient)를 계산하여 역전파를 수행합니다.

6. **파라미터 업데이트** (`update_parameters()`):
   ```python
   parameters = update_parameters(parameters, grads)
   ```
   - 역전파로 계산된 기울기를 사용해 가중치와 편향을 경사 하강법으로 업데이트합니다.

7. **비용 출력**:
   ```python
   if print_cost and i % 1000 == 0:
       print("Cost after iteration %i: %f" % (i, cost))
   ```
   - `print_cost`가 `True`일 때, 매 1000번의 반복마다 비용을 출력합니다.

### for문을 사용하는 이유:
- **반복적으로** 전방 전파 → 비용 계산 → 역전파 → 파라미터 업데이트 과정이 진행되어야 하기 때문에 `for`문을 사용합니다.
- 반복문 없이 경사 하강법을 한 번만 적용하면 모델이 충분히 학습되지 않기 때문에, `for`문으로 여러 번 경사 하강법을 적용하는 것이 필수적입니다.

따라서 이 함수는 **신경망 학습 과정**을 통합하여 `nn_model` 함수를 통해 학습하고, 학습된 파라미터를 반환하는 역할을 합니다.

In [ ]:
A2, cache = forward_propagation(X, parameters) # 전방 전파를 사용하여 A2 계산
predictions = (A2 > 0.5) # 임계값 적용: A2의 값이 0.5보다 크면 True, 그렇지 않으면 False가 되도록 합니다. 이 값을 0과 1로 변환하여 이진 분류를 수행합니다.

# 신경망 구축 과정

### 1. 패키지 가져오기

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

### 2. 데이터셋 로드

In [4]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# 평면 데이터셋 생성
X, Y = make_moons(n_samples=1000, noise=0.2, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# numpy -> torch tensor 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32).unsqueeze(1)  # Y가 (batch_size, 1) 형태로 만들어짐
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32).unsqueeze(1)

### 3. 신경망 모델 정의
PyTorch에서는 nn.Module을 상속받아 신경망을 정의합니다. PyTorch에서는 이를 클래스의 forward 메소드로 정의합니다.

In [5]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)  # 은닉층
        self.output = nn.Linear(hidden_size, output_size)  # 출력층
        self.activation = nn.Tanh()  # 활성화 함수 (tanh)
    
    def forward(self, x):
        z1 = self.hidden(x)          # 전방 전파: 은닉층
        a1 = self.activation(z1)     # 은닉층 활성화 함수 적용
        z2 = self.output(a1)         # 출력층으로 전방 전파
        return torch.sigmoid(z2)     # 이진 분류에서 sigmoid 함수 사용

## 4. 손실 함수 및 최적화기 정의
이제 손실 함수로 교차 엔트로피 손실(cross-entropy loss)을 사용하고, 최적화 알고리즘으로 경사 하강법을 사용합니다.

PyTorch에서는 torch.optim 모듈을 사용하여 이를 간편하게 처리할 수 있습니다.

In [6]:
# 모델 초기화
input_size = 2  # 데이터셋의 특성 수 (X가 2차원)
hidden_size = 4  # 은닉층 크기 (선택 사항)
output_size = 1  # 출력 크기 (이진 분류)
model = SimpleNN(input_size, hidden_size, output_size)

# 손실 함수와 최적화기 정의
criterion = nn.BCELoss()  # 이진 분류에 사용되는 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.01)  # 경사 하강법 (SGD)

### 5. 훈련 루프
이제 훈련 루프를 정의하여 데이터를 학습시키고, 매 반복마다 손실 값을 출력합니다. PyTorch는 자동 미분을 사용하여 기울기 계산을 자동으로 처리하므로, 수동으로 역전파(backpropagation)를 계산할 필요가 없습니다.

In [10]:
# 훈련 루프
num_epochs = 100000
for epoch in range(num_epochs):
    # 순전파 (forward propagation)
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)
    
    # 역전파 (backward propagation)
    optimizer.zero_grad()  # 기울기 초기화
    loss.backward()  # 역전파 자동 계산
    optimizer.step()  # 경사 하강법으로 가중치 업데이트
    
    # 100번째 에포크마다 손실 출력
    if (epoch+1) % 5000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [5000/100000], Loss: 0.2859
Epoch [10000/100000], Loss: 0.2845
Epoch [15000/100000], Loss: 0.2814
Epoch [20000/100000], Loss: 0.2585
Epoch [25000/100000], Loss: 0.1714
Epoch [30000/100000], Loss: 0.1287
Epoch [35000/100000], Loss: 0.1095
Epoch [40000/100000], Loss: 0.0995
Epoch [45000/100000], Loss: 0.0936
Epoch [50000/100000], Loss: 0.0899
Epoch [55000/100000], Loss: 0.0873
Epoch [60000/100000], Loss: 0.0854
Epoch [65000/100000], Loss: 0.0840
Epoch [70000/100000], Loss: 0.0829
Epoch [75000/100000], Loss: 0.0820
Epoch [80000/100000], Loss: 0.0812
Epoch [85000/100000], Loss: 0.0806
Epoch [90000/100000], Loss: 0.0800
Epoch [95000/100000], Loss: 0.0795
Epoch [100000/100000], Loss: 0.0791


### 6. 모델 평가
출력값이 0.5보다 큰 경우 1로, 그렇지 않은 경우 0으로 예측합니다.

In [14]:
with torch.no_grad():  # 기울기 계산을 하지 않음
    test_outputs = model(X_test)
    predicted = (test_outputs > 0.5).float()
    accuracy = (predicted == Y_test).float().mean().item()  # Boolean을 float으로 변환
    print(f'Accuracy on test data: {accuracy * 100:.2f}%')

Accuracy on test data: 97.00%


In [ ]:
# 전체 코드
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# 데이터셋 생성
X, Y = make_moons(n_samples=1000, noise=0.2, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# numpy -> torch tensor 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32).unsqueeze(1)

# 신경망 모델 정의
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, output_size)
        self.activation = nn.Tanh()

    def forward(self, x):
        z1 = self.hidden(x)
        a1 = self.activation(z1)
        z2 = self.output(a1)
        return torch.sigmoid(z2)

# 모델, 손실 함수, 최적화기 초기화
input_size = 2
hidden_size = 4
output_size = 1
model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 훈련 루프
num_epochs = 1000
for epoch in range(num_epochs):
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 모델 평가
with torch.no_grad():
    test_outputs = model(X_test)
    predicted = (test_outputs > 0.5).float()
    accuracy = (predicted == Y_test).mean().item()
    print(f'Accuracy on test data: {accuracy * 100:.2f}%')